```{contents}
```

## Guardrails (LangChain Perspective)


**Guardrails** are **controls applied before, during, and after LLM execution** to ensure outputs are **safe, correct, compliant, and reliable**.

> Guardrails do not improve creativity; they **reduce risk**.

In LangChain, guardrails are implemented as **composable layers** using prompts, validators, output parsers, retries, filters, and routing.

---

### Why Guardrails Are Necessary

LLMs can:

* Hallucinate facts
* Ignore instructions
* Leak sensitive data
* Produce unsafe content
* Return invalid formats

Guardrails:

* Enforce structure
* Validate outputs
* Block unsafe inputs
* Provide fallbacks
* Increase determinism

---

### Where Guardrails Sit in the Pipeline

```
User Input
  ↓
Input Guardrails
  ↓
Prompt + LLM
  ↓
Output Guardrails
  ↓
Validated Response
```

LangChain lets you place guardrails **at every stage**.

---

### Types of Guardrails

#### Input Guardrails

* Prompt injection detection
* Input sanitization
* Length / schema validation

#### Generation Guardrails

* Low temperature
* Tool-only responses
* Restricted instructions

#### Output Guardrails

* Structured output validation
* Content filters
* Auto-fixing and retries
* Fallback routing

---

### Input Guardrails (Demonstration)

#### Basic Input Validation



In [ ]:
def validate_input(text: str):
    if "ignore previous instructions" in text.lower():
        raise ValueError("Potential prompt injection detected")
    return text




### Using RunnableLambda



In [ ]:
from langchain_core.runnables import RunnableLambda

input_guard = RunnableLambda(validate_input)

safe_chain = input_guard | prompt | llm




---

### Prompt-Level Guardrails

#### Strong System Message



In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Follow system rules. Do not reveal internal reasoning."),
    ("human", "{input}")
])




System messages act as **soft guardrails**.

---

### Output Guardrails with Structured Output (Primary)

#### Schema Enforcement (Best Practice)



In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI

class Ticket(BaseModel):
    category: str
    priority: str

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

guarded_llm = llm.with_structured_output(Ticket)

result = guarded_llm.invoke("Email service is down for finance team")
print(result)




Guarantees:

* Correct fields
* Correct types
* Retry on failure

---

### Output Validation + Auto-Fix

#### OutputFixingParser



In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser

parser = PydanticOutputParser(pydantic_object=Ticket)
fixing_parser = OutputFixingParser.from_llm(llm, parser)




If the model returns invalid JSON:

* LangChain re-prompts
* Repairs output automatically

---

### Content Safety Guardrails

#### Simple Keyword Filter



In [ ]:
def safety_filter(output: str):
    blocked = ["password", "secret"]
    if any(b in output.lower() for b in blocked):
        raise ValueError("Sensitive content detected")
    return output

output_guard = RunnableLambda(safety_filter)
safe_chain = prompt | llm | output_guard




---

### Guardrails with Retry & Fallback

#### Retry on Failure



In [ ]:
safe_chain = (prompt | llm).with_retry(max_attempts=3)




### Fallback Model



In [ ]:
safe_chain = (prompt | llm).with_fallbacks([backup_chain])




Used when:

* Model fails validation
* Rate limits occur
* Output is unsafe

---

### Guardrails in RAG

#### Context-Only Answers



In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer only using the provided context."),
    ("human", "Context:\n{context}\n\nQuestion:\n{input}")
])


Combined with:

* Low temperature
* Source citation schema

---

### Guardrails in Agents

#### Tool-Only Execution

* Tools have strict JSON schemas
* Agent cannot invent tool arguments
* Invalid tool calls are rejected

Agents rely on:

* Structured tool schemas
* AgentExecutor validation
* Hidden chain-of-thought

---

### Common Guardrail Patterns

#### Allowlist

* Allowed tools
* Allowed output labels
* Allowed fields

#### Denylist

* Prompt injection phrases
* Sensitive keywords

#### Validation

* Schema checks
* Range checks
* Enum checks

---

### Common Mistakes

#### Relying only on prompts

❌ Soft guardrails are not enough

#### Overly complex schemas

❌ Increases failure rate

#### High temperature with guardrails

❌ More retries and cost

#### No fallback strategy

❌ Fragile systems

---

### Best Practices

* Use **structured output** as the primary guardrail
* Keep temperature low for deterministic tasks
* Validate both input and output
* Add retries and fallbacks
* Never expose chain-of-thought
* Log guardrail failures internally

---

### When Guardrails Are Mandatory

| Scenario      | Guardrails |
| ------------- | ---------- |
| APIs          | Required   |
| Agents        | Required   |
| RAG           | Required   |
| Automation    | Required   |
| Creative chat | Optional   |

---

### Interview-Ready Summary

> “Guardrails in LangChain are layered controls—input validation, prompt constraints, structured output enforcement, retries, and fallbacks—that ensure LLM outputs are safe, valid, and production-ready.”

---

### Rule of Thumb

* **If a machine consumes the output → guard it**
* **If it’s production → guard it twice**
* **If it’s safety-critical → validate everything**